In [ ]:
'''
Importing relevant libraries for project.
'''
import pandas as pd
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
'''
Importing CSV file of valid postcodes as a list.
'''
path = 'C:/Users/coran.mulcahy/Documents/Coran_personal/git_repositories/ga_python_project/Postcodes/POA_2016_aust.csv'
POA_2016 = open(path)
valid_postcodes = csv.reader(POA_2016)

for row in valid_postcodes:
    valid_postcodes = row

In [ ]:
print(valid_postcodes)

In [ ]:
"""
Definition of functions for script
"""

def percent_progress():
    progress = float(counter)/len(valid_postcodes)*100
    return (f"Progress: {round(progress,1)}%")

def dropdown_loop():
    '''
    The Foodworks website will only load one store at a time. So, when inputting every postcode, there is a chance that a Foodworks will be missed
    if there is multiple Foodworks in the same postcode (eg. Fitzroy). This code, will create a list of all the dropdown suggestions and itrate through 
    them before moving on to the next postcode.
    '''
    dropdown_stores = [ ]

    for dropdown in browser.find_elements_by_xpath(".//div[@class='nav-search-result-item']"):
        dropdown_stores.append(dropdown.text)
    '''
    This loop does some text cleaning for input into search box (eg. "Bentleigh - 3204" becomes "Bentleigh" )
    '''
    dropdown_cleaned = [ ]

    for dropdown in dropdown_stores:
        dropdown_cleaned.append(dropdown[: -7])
    '''
    Take the suggested dropdown stores into the search box and scrape their addresses
    '''
    for dropdown in dropdown_cleaned:

        search_box = browser.find_element_by_id("navigation_store_search")
        search_box.click()
        search_box.clear()
        search_box.send_keys(dropdown)

        time.sleep(5)

        new_dropdown = browser.find_element_by_xpath("//div[@class='nav-search-result-item']/span[1]")
        new_dropdown.click()

        time.sleep(5)

        """
        Selenium will now scrape the store name and store address for the searched store. The resulting text is then cleaned
        before being stored in lists.
        """
        store_name = [ ]
        store_address = [ ]

        for store in browser.find_elements_by_xpath('.//h1[@class = "theme-heading theme-spacing-sm"]'):
            store_name.append(store.text)

        for address in browser.find_elements_by_xpath('.//address[@class = "theme-body theme-bold theme-spacing-sm"]'):
            store_address.append(address.text)

        cleaned_store_address_list = [ ]
        cleaned_store_name_list = [ ]

        for store in store_address:
            cleaned_store_address_list.append(store.replace('\n' , ', '))

        for store in store_name:
            cleaned_store_name_list.append(store.replace('WELCOME TO FOODWORKS\n' , 'Foodworks ').title())

        """
        Iterate through the two lists of store name and store address and add them to the master dictionary that we created earlier.
        """
        for i in range(len(store_name)):
            foodworks_stores.update( {cleaned_store_name_list[i] : cleaned_store_address_list[i]} )

        find_store_button = browser.find_element_by_class_name("nav-link")
        find_store_button.click()
    


In [ ]:
'''
Creating a counter for number of iterations and an empty master dictionary for scraped Foodworks store data to be appended to later on.
As well as a function for progress.
'''
counter = 0
foodworks_stores = { }

'''
Selenium set up and navigation to Foodworks website.
'''
browser = webdriver.Chrome()
browser.get('https://foodworks.com.au/')

'''
Selenium is being used to simulate a user navigating through the Foodworks website and inputting postcodes into the search box.

Selenium will:
Navigate to the search box, clear text (if there is any) from the search box, input a postcode, wait for the suggestions box to load, press the down arrow
key and then enter key to select the first suggestion of the dropdown list.
      
This code is within a for loop, to iterate through the list of valid Australian postcodes.
'''
for postcode in valid_postcodes:
    
    time.sleep(5)
    
    find_store_button = browser.find_element_by_class_name("nav-link")
    find_store_button.click()
    
    time.sleep(5)
    
    search_box = browser.find_element_by_id("navigation_store_search") 
    search_box.click()
    search_box.send_keys(postcode)
    
    time.sleep(5)
        
    dropdown_suggestions = browser.find_elements_by_xpath("//div[@class='nav-search-result-item']/span")
    
    if (dropdown_suggestions):   
            
        dropdown_loop()
          
        home_button = browser.find_element_by_class_name("navbar-brand")
        home_button.click()
                            
        counter += 1
        print(f"Store suggestions for postcode {postcode} have been added")
        print(f"Progress: {percent_progress()}")
              
    else:
        counter += 1
        print(f"No stores found for {postcode}, progressing to next postcode ({percent_progress()})")
        search_box.click()
        search_box.clear()
        

In [ ]:
foodworks_stores

In [ ]:
'''
Finally, take the fully populated master dictionary and output to a .csv file.
'''
(pd.DataFrame.from_dict(data = foodworks_stores, orient = 'index').to_csv('scraped_NSW_Foodworks_stores.csv', header=False))